In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
import plotly.express as px
from sklearn import linear_model

# Load Data

In [46]:
train = pd.read_csv("data/train/train.csv").set_index("id")
test = pd.read_csv("data/test/test.csv").set_index("id")

# Preprocesser

In [36]:
class preprocessor:
    def __init__(self):
        a = 1
    #def fit(self, data):
    
    def fit_transform(self, data):
        #y = data["price"]
        #x = data.drop("price", axis = 1)
        cate_feature = data.dtypes
        self.cate_feature = list(cate_feature[cate_feature == "object"].index)
        #print('cate_feature' ,self.cate_feature )
        x = data
        x["year"][x["year"]> 2024] = x["year"][x["year"]> 2024] - 1000
        x["odometer"][x["odometer"]< -2] = np.abs(x["odometer"][x["odometer"]< -2])
        x = x.drop(x.loc[x['odometer']>1000000].index)
        self.odometer_mean =  train[['condition' , 'odometer']].groupby('condition').mean()
        for index_ in self.odometer_mean.index:
            x.loc[ (x['odometer'] ==-1) & (x['condition'] ==index_),'odometer'  ] = self.odometer_mean.loc[index_].values[0]
#         x["odometer"][x["odometer"]> 5e+5] = 4.5e+5
        #x["odometer"][x["odometer"]==-1] = pd.NA
        
#         x["manufacturer"] = (x["manufacturer"].str.normalize("NFKC")).str.lower()
        symbols = (u"αабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯѕᴄ",
               u"aabvgdeejzijklmnoprstufhzcss_y_euaABVGDEEJZIJKLMNOPRSTUFHZCSS_Y_EUAsc")
        tr = {ord(a):ord(b) for a, b in zip(*symbols)}
        x['manufacturer'] = x['manufacturer'].map(lambda x : unicodedata.normalize('NFKC', x).lower().translate(tr))
        x['manufacturer'] = x['manufacturer'].replace({'land rover':'rover'})
        
        
        self.cylinders = pd.Series([6,4,8, -1, 10,5,3,12], index = x["cylinders"].value_counts().index)
        x["cylinders"] = self.cylinders[x["cylinders"]].values
        
        tmp = x["fuel"].value_counts()
        self.fuel = pd.Series(tmp.index, index = tmp.index)
        #self.fuel["other"] = pd.NA
        x["fuel"] = self.fuel[x["fuel"].fillna("other")].values
        
        tmp = x["size"].value_counts()
        self.size = pd.Series([t[0] for t in tmp.index], index = tmp.index)
        x["size"] = self.size[x["size"].fillna("other")].values
        
        x[self.cate_feature] = x[self.cate_feature].fillna("other")
        
        #x[self.cate_feature] = x[self.cate_feature].astype('category')
        #print('self.cate_feature',self.cate_feature)
#         condition_rank = {
#                 'salvage' : 1,
#                 'fair' : 2,
#                 'good' : 3, 
#                 'excellent': 4,
#                 'like new' : 5, 
#                 'new' : 6
#             }
#         x['condition_rank'] = x['condition'].replace(condition_rank)
        
        self.label_emb = {}
        for feature in self.cate_feature:
            self.label_emb[feature] = x[[feature, "price"]].groupby(feature).mean()["price"]
        #print('self.label_emb',self.label_emb) #-----------------------------------------------------
        for feature in self.cate_feature:
            x[feature] = self.label_emb[feature].loc[x[feature]].values
        #print(x)
        # Global feature
        counts = x.groupby("year").size()
        counts = counts/counts.sum()
        x["counts"] = counts[x["year"]].values
        
        self.find_year_price_model(x["year"], x["price"])
        x["year_avg"] = self.yp.predict(self.power((x["year"]-1970)/50))
        #self.find_year_price_model(x["odometer"], x["price"])
        #x['odometer_avg'] = self.yp1.predict(self.power((x["odometer"]-200000)/200000))
        #x = x.drop(columns = 'region')
        return x
    
    def power(self, y, n = 4):
        return np.stack([y**i for i in range(4)], axis = -1)
    
    def find_year_price_model(self, t, p):
        self.yp = linear_model.Ridge(alpha=.01)
        t = (t-1970)/50
        t = self.power(t)
        self.yp.fit(t, p)
        
    def find_odemeter_price_model(self, t, p):
        self.yp = linear_model.Ridge(alpha=.01)
        t = (t-200000)/200000
        t = self.power(t)
        self.yp1.fit(t, p)
        
    def expand(self, feature_mapping, test_data):
        return pd.DataFrame([test_data.value_counts(), feature_mapping], index = ["test", "train"]).T["train"]
        
    def transform(self, data):
        #y = data["price"]
        #x = data.drop("price", axis = 1)
        
        x = data
        x["year"][x["year"]> 2024] = x["year"][x["year"]> 2024] - 1000
        x["odometer"][x["odometer"]< -2] = np.abs(x["odometer"][x["odometer"]< -2])
        x["odometer"][x["odometer"]> 5e+5] = 4.5e+5
        #x["odometer"][x["odometer"]==-1] = pd.NA
        for index_ in self.odometer_mean.index:
            x.loc[ (x['odometer'] ==-1) & (x['condition'] ==index_),'odometer'  ] = self.odometer_mean.loc[index_].values[0]
            
        #x["manufacturer"] = (x["manufacturer"].str.normalize("NFKC")).str.lower()
        symbols = (u"αабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯѕᴄ",
               u"aabvgdeejzijklmnoprstufhzcss_y_euaABVGDEEJZIJKLMNOPRSTUFHZCSS_Y_EUAsc")
        tr = {ord(a):ord(b) for a, b in zip(*symbols)}
        x['manufacturer'] = x['manufacturer'].map(lambda x : unicodedata.normalize('NFKC', x).lower().translate(tr))
        x['manufacturer'] = x['manufacturer'].replace({'land rover':'rover'})
        
        #self.cylinders = pd.Series([6,4,8, -1, 10,5,3,12], index = x["cylinders"].value_counts().index)
        self.cylinders = self.expand(self.cylinders, x["cylinders"])
        x["cylinders"] = self.cylinders[x["cylinders"]].values
        
        #tmp = x["fuel"].value_counts()
        #self.fuel = pd.Series(tmp.index, index = tmp.index)
        #self.fuel["other"] = pd.NA
        self.fuel = self.expand(self.fuel, x["fuel"].fillna("other"))
        x["fuel"] = self.fuel[x["fuel"].fillna("other")].values
        
        #tmp = x["size"].value_counts()
        #self.size = pd.Series([t[0] for t in tmp.index], index = tmp.index)
        self.size = self.expand(self.size, x["size"].fillna("other"))
        x["size"] = self.size[x["size"].fillna("other")].values
        
        x[self.cate_feature] = x[self.cate_feature].fillna("other")
        
        #x[self.cate_feature] = x[self.cate_feature].astype('category')
        #self.label_emb = {}
        #for feature in self.cate_feature:
         #   self.label_emb[feature] = x[[feature, "price"]].groupby(feature).mean()
#         condition_rank = {
#                 'salvage' : 1,
#                 'fair' : 2,
#                 'good' : 3, 
#                 'excellent': 4,
#                 'like new' : 5, 
#                 'new' : 6
#             }
#         x['condition_rank'] = x['condition'].replace(condition_rank)
        for feature in self.cate_feature:
            self.label_emb[feature] = self.expand(self.label_emb[feature], x[feature])
            x[feature] = self.label_emb[feature].loc[x[feature]].values
        
        # Global feature
        counts = x.groupby("year").size()
        counts = counts/counts.sum()
        x["counts"] = counts[x["year"]].values
        
        #self.find_year_price_model(x["year"], x["price"])
        x["year_avg"] = self.yp.predict(self.power((x["year"]-1970)/50))
#         x['odometer_avg'] = self.yp1.predict(self.power((x["odometer"]-200000)/200000))
        #x = x.drop(columns = 'region')
        return x
pp = {}
pp["cv1"] = preprocessor()
    
train_x = pp["cv1"].fit_transform(train)
test_x = pp["cv1"].transform(test)  

# CV

In [38]:
from sklearn.model_selection import KFold

kf = KFold(5, shuffle= True)
cvs = kf.split(train)

# Parameter

In [39]:
params = {}

params['subsample_for_bin'] = 20000
params['learning_rate'] = 1e-2
params['max_bin'] = 1024

params['n_estimators'] = 1024
params['boosting_type'] = 'gbdt'
params['objective'] = 'mape'
params['sub_feature'] = 0.8      # feature_fraction 
params['bagging_fraction'] = 0.8 # sub_row
params['bagging_freq'] = 8


params['num_leaves'] = 128-1        # num_leaf
params['max_depth'] = 7
params['min_data'] = 4         # min_data_in_leaf

params['min_hessian'] = 1e-4     # min_sum_hessian_in_leaf
params['verbose'] = 0
params["early_stopping_rounds"] = 20

params["reg_lambda"] = 1e-3



# Train

In [ ]:
i=0
result = {}
models = {}

for train_ind, valid_ind in cvs:
    pp["cv"+str(i)] = preprocessor()
    
    train_x = pp["cv"+str(i)].fit_transform(train.iloc[train_ind])
    train_y = train_x["price"]
    train_x = train_x.drop("price", axis= 1)
    
    valid_x = pp["cv"+str(i)].transform(train.iloc[valid_ind])
    valid_y = valid_x["price"]
    valid_x = valid_x.drop("price", axis= 1)
    
    
    models["cv"+str(i)] = lgb.LGBMRegressor(**params)
    result["cv"+str(i)] = models["cv"+str(i)].fit(train_x, train_y, eval_set= (valid_x, valid_y))
    
    
    i+=1

# Predict

In [17]:
predictions = {}
for i in range(5):
    x = pp["cv"+str(i)].transform(test.copy())
    predictions["cv"+str(i)] = models["cv"+str(i)].predict(x)

C:\Users\USER\AppData\Local\Temp\ipykernel_1220\3876202847.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["year"][x["year"]> 2024] = x["year"][x["year"]> 2024] - 1000
C:\Users\USER\AppData\Local\Temp\ipykernel_1220\3876202847.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["odometer"][x["odometer"]< -2] = np.abs(x["odometer"][x["odometer"]< -2])
C:\Users\USER\AppData\Local\Temp\ipykernel_1220\3876202847.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

[LightGBM] [Warning] feature_fraction is set with colsample_bytree=1.0, will be overridden by sub_feature=0.8. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_data=4. Current value: min_data_in_leaf=4
[LightGBM] [Warning] min_sum_hessian_in_leaf is set with min_child_weight=0.001, will be overridden by min_hessian=0.0001. Current value: min_sum_hessian_in_leaf=0.0001
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8


C:\Users\USER\AppData\Local\Temp\ipykernel_1220\3876202847.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["odometer"][x["odometer"]< -2] = np.abs(x["odometer"][x["odometer"]< -2])
C:\Users\USER\AppData\Local\Temp\ipykernel_1220\3876202847.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["odometer"][x["odometer"]> 5e+5] = 4.5e+5
C:\Users\USER\AppData\Local\Temp\ipykernel_1220\3876202847.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

[LightGBM] [Warning] feature_fraction is set with colsample_bytree=1.0, will be overridden by sub_feature=0.8. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_data=4. Current value: min_data_in_leaf=4
[LightGBM] [Warning] min_sum_hessian_in_leaf is set with min_child_weight=0.001, will be overridden by min_hessian=0.0001. Current value: min_sum_hessian_in_leaf=0.0001
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set with colsample_bytree=1.0, will be overridden by sub_feature=0.8. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_data=4. Current value: min_data_in_leaf=4
[LightGBM] [Warning] min_sum_hes

C:\Users\USER\AppData\Local\Temp\ipykernel_1220\3876202847.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["year"][x["year"]> 2024] = x["year"][x["year"]> 2024] - 1000
C:\Users\USER\AppData\Local\Temp\ipykernel_1220\3876202847.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["odometer"][x["odometer"]< -2] = np.abs(x["odometer"][x["odometer"]< -2])
C:\Users\USER\AppData\Local\Temp\ipykernel_1220\3876202847.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

[LightGBM] [Warning] feature_fraction is set with colsample_bytree=1.0, will be overridden by sub_feature=0.8. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_data=4. Current value: min_data_in_leaf=4
[LightGBM] [Warning] min_sum_hessian_in_leaf is set with min_child_weight=0.001, will be overridden by min_hessian=0.0001. Current value: min_sum_hessian_in_leaf=0.0001
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] feature_fraction is set with colsample_bytree=1.0, will be overridden by sub_feature=0.8. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_data=4. Current value: min_data_in_leaf=4
[LightGBM] [Warning] min_sum_hes

In [22]:
test["result"] = np.array([predictions["cv"+str(i)] for i in range(5)]).mean(axis = 0)